In [1]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
from itables import init_notebook_mode, show
from collections import Counter
import networkx as nx
from pyvis.network import Network
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from collections import defaultdict
from sklearn import metrics
from time import time

## Exercice personnel de clusterisation de textes


* code issu de cet exemple: https://scikit-learn.org/stable/auto_examples/text/plot_document_clustering.html

### Application à mes données —— commence ici

In [3]:
f = "book_titles.csv"
txt_astro = pd.read_csv(f, sep="|")

In [4]:
txt_astro.head()

,pk_idref_books,book_date,simplified_title
0,1,2001,Structure interpretation classical mechanics
1,2,1926,motion distribution stars
2,3,1968,Catalog emission lines astrophysical objects
3,4,1969,Catalog emission lines astrophysical objects
4,5,1976,Applied solar energy introduction


In [5]:
txt_astro.simplified_title[-10:-5]

5521                    rôle Société Astronomique France
5522                                     أجمل تاريخ للكون
5523    Comment pensent savants leçons grands scientif...
5524                                   Hermann Carl Vogel
5525    L'Université l'usine (1956-1984) fondateur L...
Name: simplified_title, dtype: object

### Not in use


In [6]:
def ab(text):
    for ch in ['\\','\n','?','*','{','}','[',']','(',')','>','#',';',':','.','!','$','\'']:
        text = text.replace(ch,"")
    return text    

In [7]:
print(ab('aéslfk? js#aélfk: jaésd$d,'))

aéslfk jsaélfk jaésdd,


In [8]:
def suppr_math(text):
    for ch in ['mathematics','mathematical','mathematician']:
        text = text.replace(ch,"")
    return text   

In [9]:
txt_astro.head(2)

,pk_idref_books,book_date,simplified_title
0,1,2001,Structure interpretation classical mechanics
1,2,1926,motion distribution stars


### Follows from here the code

In [10]:
txt_astro['title'] = txt_astro['simplified_title'].apply(lambda x : x.lower().strip())

In [11]:
txt_astro.title[-10:-5]

5521                    rôle société astronomique france
5522                                     أجمل تاريخ للكون
5523    comment pensent savants leçons grands scientif...
5524                                   hermann carl vogel
5525    l'université l'usine (1956-1984) fondateur l...
Name: title, dtype: object

In [12]:
txt_astro['txt_len'] = txt_astro['title'].apply(lambda x : len(x))

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    max_df=0.5,
    min_df=5,
    stop_words="english",
)

In [14]:
t0 = time()
X_tfidf = vectorizer.fit_transform(txt_astro.title)

print(f"vectorization done in {time() - t0:.3f} s")
print(f"n_samples: {X_tfidf.shape[0]}, n_features: {X_tfidf.shape[1]}")

vectorization done in 0.048 s
n_samples: 5531, n_features: 1699


In [ ]:
### https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
vectorizer.get_feature_names_out()[:5]

In [ ]:
vectorizer.ngram_range

In [17]:
? X_tfidf

Type:        csr_matrix
String form:
(0, 949)	0.5170463345367282
           (0, 824)	0.6881376702785902
           (0, 1491)	0.5090477724985034
           (1,  <...> 5529, 1442)	0.5873142875592519
           (5530, 1316)	0.7623685483909642
           (5530, 648)	0.6471431035128582
File:        ~/miniconda3/envs/py310_data_analysis/lib/python3.10/site-packages/scipy/sparse/_csr.py
Docstring:  
Compressed Sparse Row matrix

This can be instantiated in several ways:
    csr_matrix(D)
        with a dense matrix or rank-2 ndarray D

    csr_matrix(S)
        with another sparse matrix S (equivalent to S.tocsr())

    csr_matrix((M, N), [dtype])
        to construct an empty matrix with shape (M, N)
        dtype is optional, defaulting to dtype='d'.

    csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])
        where ``data``, ``row_ind`` and ``col_ind`` satisfy the
        relationship ``a[row_ind[k], col_ind[k]] = data[k]``.

    csr_matrix((data, indices, indptr), [shape=(M, N)])
 

In [18]:
X_tfidf.data,X_tfidf.indices,X_tfidf.data 

(array([0.51704633, 0.68813767, 0.50904777, ..., 0.58731429, 0.76236855,
        0.6471431 ]),
 array([ 949,  824, 1491, ..., 1442, 1316,  648], dtype=int32),
 array([0.51704633, 0.68813767, 0.50904777, ..., 0.58731429, 0.76236855,
        0.6471431 ]))

In [19]:
print(f"{X_tfidf.nnz / np.prod(X_tfidf.shape):.3f}")

0.003


In [ ]:
from sklearn.cluster import KMeans

for seed in range(5):
    kmeans = KMeans(
        n_clusters=20,
        max_iter=500,
        n_init=50,
        random_state=seed,
    ).fit(X_tfidf)
    cluster_ids, cluster_sizes = np.unique(kmeans.labels_, return_counts=True)
    print(f"Number of elements asigned to each cluster: {cluster_sizes}")
print()


In [21]:
y_tfidf = kmeans.predict(X_tfidf)

In [ ]:
y_tfidf[:10]

In [ ]:
lines_classes =  ['_'+ str(i+1) if i < 9 else str(i+1) for i in y_tfidf]; lines_classes[:3]

In [ ]:
len(lines_classes), len(txt_astro)

In [30]:

# To see top terms per cluster
terms = vectorizer.get_feature_names_out()
for i in range(20):
    cluster_center = kmeans.cluster_centers_[i]
    top_terms_idx = cluster_center.argsort()[-10:][::-1]  # Top 10 terms
    top_terms = [terms[idx] for idx in top_terms_idx]
    print(f"Cluster {i}: {', '.join(top_terms)}")

Cluster 0: theory, relativity, general, special, meaning, heat, mathematical, gases, lunar, gravitation
Cluster 1: universe, life, bang, complete, nature, mysterious, cosmos, expanding, creation, history
Cluster 2: univers, histoire, bang, trous, noirs, évolution, discussion, temps, il, brève
Cluster 3: einstein, albert, collected, volume, works, papers, correspondence, years, berlin, writings
Cluster 4: stars, catalogue, variable, proper, motions, cometary, orbits, spectra, double, fixed
Cluster 5: jahren, beobachtungen, sternwarte, 1819, nach, 1816, angestellten, kaiserlichen, 1821, sternen
Cluster 6: travaux, notice, scientifiques, titres, monge, supplément, bigourdan, rapport, exposé, essai
Cluster 7: relativité, théorie, restreinte, générale, espace, problème, quanta, théories, premiers, concepts
Cluster 8: stellar, soleil, étoiles, galaxies, band, physique, mars, werke, note, recherches
Cluster 9: james, maxwell, clerk, paisley, poet, john, blind, scientific, papers, poem
Cluster

In [25]:
txt_astro.head()

,pk_idref_books,book_date,simplified_title,title,txt_len
0,1,2001,Structure interpretation classical mechanics,structure interpretation classical mechanics,44
1,2,1926,motion distribution stars,motion distribution stars,25
2,3,1968,Catalog emission lines astrophysical objects,catalog emission lines astrophysical objects,44
3,4,1969,Catalog emission lines astrophysical objects,catalog emission lines astrophysical objects,44
4,5,1976,Applied solar energy introduction,applied solar energy introduction,33


In [26]:
f = "book_titles_clusters.csv"
txt_astro.to_csv(f, sep='|')

In [27]:
txt_astro['class_search'] = lines_classes; txt_astro['class'] = y_tfidf 

In [ ]:
### https://github.com/mwouts/itables/blob/main/docs/advanced_parameters.md
show(txt_astro.sort_values(by='txt_len'), classes="display", scrollY="500px", 
     scrollCollapse=True, paging=False, column_filters="footer", dom="lrtip")

### not used here

In [ ]:
txt_astro['tokens'] = txt_astro['summ_repl'].apply(lambda x : x.split(' '))

In [ ]:
dfg = pd.DataFrame(txt_astro.groupby('class')['tokens'].apply(list))
dfg.columns = ['tokens']
dfg = dfg.reset_index()
dfg.head(2)

In [ ]:
def most_frequent_tokens(l):
    sl = sum(l, [])
    kl = [tk.replace('\n', '') \
          #.replace('.', '').replace(',', '').strip().lower() \
          for tk in sl if len(tk)>3]
          #for tk in sl if len(tk)>3 and tk not in ('mathematician','worked', 'work', 'known')]
    skl = pd.Series(Counter(kl))
    fskl = skl.loc[lambda x : x > 3].sort_values(ascending=False)
    final_l = fskl.iloc[:20].to_dict()
    
    return final_l

In [ ]:
dfg['mf_tokens'] = dfg['tokens'].apply(lambda x : most_frequent_tokens(x))

In [ ]:
### https://github.com/mwouts/itables/blob/main/docs/advanced_parameters.md
show(dfg, classes="display", scrollY="400px", 
     scrollCollapse=True, paging=False, column_filters="footer", dom="lrtip")

### Produce graph

In [ ]:
fl = dfg['mf_tokens'][0]; fl

In [ ]:
l = dfg['tokens'][1][0]; l[:7]

In [ ]:
[i for i in l if i in fl.keys()]

In [ ]:
sorted(['William', 'Morgan', 'was', 'an', 'early', 'actuary'])

In [ ]:
l =  ['Morgan', 'William', 'actuary', 'an', 'early', 'was']

In [ ]:
def tks_graph(l, fl):
    lfk = list(fl.keys())

    le_dct= []
    for e in l: 
        l_dct= []
        sl = sorted([i for i in e if len(i) > 3 and i in lfk ])
        for i in sl:
            for v in sl:
                if i < v:
                    l_dct.append([i,v])
        #print(l_dct)
        le_dct += l_dct            
    return le_dct

In [ ]:
dfg['tokens_graph'] = dfg.apply(lambda x : tks_graph(x.tokens,x.mf_tokens), axis=1)
dfg.head()

In [ ]:
dfg.tokens_graph[:3]

In [ ]:
ll = [tuple(e) for e in dfg.loc[0]['tokens_graph']]; # ll

In [ ]:
fl = dfg['mf_tokens'][0]; fl

In [ ]:
d_eff = dict([(k, {'eff':v}) for k,v in fl.items()]); d_eff

In [ ]:
G = nx.Graph()
G.add_edges_from(ll)

In [ ]:
G.nodes()

In [ ]:
nx.set_node_attributes(G, d_eff)

# inspecter les sommets et leurs attributs
list(G.nodes.data())

In [ ]:
nx.get_node_attributes(G, 'eff').values()

In [ ]:
[i]

In [ ]:
pos = nx.kamada_kawai_layout(G)
fig = plt.figure(figsize = (12,9))
fig.tight_layout()
# nx.draw_networkx(G, pos, node_size=10)
nx.draw_networkx_edges(G, pos, alpha=0.1)
nx.draw_networkx_labels(G, pos, horizontalalignment='left', verticalalignment='bottom')
nx.draw_networkx_nodes(G, pos, node_size= list(nx.get_node_attributes(G, 'eff').values()))


In [ ]:
# https://engineeringfordatascience.com/posts/matplotlib_subplots/
# https://matplotlib.org/stable/gallery/subplots_axes_and_figures/subplot.html#sphx-glr-gallery-subplots-axes-and-figures-subplot-py


plt.subplots_adjust( hspace=10)
ncols = 1
# calculate number of rows
nrows = len(dfg) 

i = 0






while i < nrows :

    val = pd.DataFrame.from_dict(dfg.iloc[i]['mf_tokens'], orient='index')
    val.columns=['eff']
    val['y_val'] = [1]*len(val)
    # val['ln_eff'] = val['eff'].apply(lambda x : np.log(x)) ça n'apporte rien


    # print(val.eff)
    
    # ax= plt.subplot(nrows,1,i+1)
    
    ax = plt.figure(figsize=(14,3))



    ax = sns.scatterplot(val, x = 'eff', y = 'y_val', alpha = 0.5)
    
    # plt.xlabel(val['eff'].to_dict(), fontsize=14, wrap=True)
    
    ax.collections[0].set_sizes(val.eff**2) 

    ax.set_xlim(min(val.eff)- min(val.eff)/10, max(val.eff)+max(val.eff)/20)
    ax.set_ylim(0, 2)
    ax.invert_xaxis()

    #For each point, we add a text inside the bubble
    for line in range(0,val.shape[0]):
         ax.text(val.eff[line], val.y_val[line]+ 0.2, str(val.index[line]).replace('{}', ''),
                 rotation=45,horizontalalignment='center', 
                 size=10, color='black') # , weight='semibold'

    ax.set_title(f'Cluster {i}', fontsize=18)

    plt.tight_layout()   

    plt.show()
    
    
    
    ### 
    ll = [tuple(e) for e in dfg.iloc[i]['tokens_graph']]
    fl = dfg['mf_tokens'][i]
    d_eff = dict([(k, {'eff':v}) for k,v in fl.items()])
    G = nx.Graph()
    G.add_edges_from(ll)
    nx.set_node_attributes(G, d_eff)
    
    d = sorted(nx.get_node_attributes(G, 'eff').items(), key=lambda x: x[1], reverse=True)
    print(d)
    
    pos = nx.kamada_kawai_layout(G)
    fig = plt.figure(figsize = (6,4))
    fig.tight_layout()
    # nx.draw_networkx(G, pos, node_size=10)
    nx.draw_networkx_edges(G, pos, alpha=0.1)
    nx.draw_networkx_labels(G, pos, horizontalalignment='left', verticalalignment='bottom')
    nx.draw_networkx_nodes(G, pos, node_size= list(nx.get_node_attributes(G, 'eff').values()))


    ax.set_title(f'Cluster {i}', fontsize=18)

    plt.tight_layout()   
    i = i + 1
    plt.show()


    


### Les 20 mots les plus fréquents par classe

In [ ]:
# https://engineeringfordatascience.com/posts/matplotlib_subplots/
# https://matplotlib.org/stable/gallery/subplots_axes_and_figures/subplot.html#sphx-glr-gallery-subplots-axes-and-figures-subplot-py


plt.subplots_adjust( hspace=10)
ncols = 1
# calculate number of rows
nrows = len(dfg) 

i = 0






while i < nrows :

    ### 
    val = pd.DataFrame.from_dict(dfg.iloc[i]['mf_tokens'], orient='index')
    val.columns=['eff']
    val['y_val'] = [1]*len(val)
    # val['ln_eff'] = val['eff'].apply(lambda x : np.log(x)) ça n'apporte rien


    # print(val.eff)
    
    # ax= plt.subplot(nrows,1,i+1)
    
    ax = plt.figure(figsize=(30,6))



    ax = sns.scatterplot(val, x = 'eff', y = 'y_val', alpha = 0.5)
    
    plt.xlabel(val['eff'].to_dict(), fontsize=14, wrap=True)
    
    ax.collections[0].set_sizes(val.eff**2) 

    ax.set_xlim(min(val.eff)- min(val.eff)/10, max(val.eff)+max(val.eff)/20)
    ax.set_ylim(0, 2)
    ax.invert_xaxis()

    #For each point, we add a text inside the bubble
    for line in range(0,val.shape[0]):
         ax.text(val.eff[line], val.y_val[line]+ 0.2, str(val.index[line]).replace('{}', ''),
                 rotation=45,horizontalalignment='center', 
                 size=16, color='black') # , weight='semibold'

    ax.set_title(f'Cluster {i}', fontsize=18)

    plt.tight_layout()   
    i = i + 1
    plt.show()


    


## Préparation 

In [ ]:
val = pd.DataFrame.from_dict((dfg.iloc[7].values)[0], orient='index'); type(val)
val.columns=['eff']
val['y_val'] = [1]*len(val)
# val['ln_eff'] = val['eff'].apply(lambda x : np.log(x)) ça n'apporte rien


In [ ]:
fig = plt.figure(figsize=(20, 4))


val = pd.DataFrame.from_dict((dfg.iloc[0].values)[0], orient='index'); type(val)
val.columns=['eff']
val['y_val'] = [1]*len(val)
# val['ln_eff'] = val['eff'].apply(lambda x : np.log(x)) ça n'apporte rien


ax = sns.scatterplot(val, x = 'eff', y = 'y_val', alpha = 0.5)
ax.set(xlabel=val['eff'].to_dict())
ax.collections[0].set_sizes(val.eff**2) 

ax.set_xlim(min(val.eff)- min(val.eff)/10, max(val.eff)+max(val.eff)/20)
ax.set_ylim(0, 2)
ax.invert_xaxis()

#For each point, we add a text inside the bubble
for line in range(0,val.shape[0]):
     ax.text(val.eff[line], val.y_val[line]+ 0.2, str(val.index[line]).replace('{}', ''),
             rotation=45,horizontalalignment='center', 
             size=12, color='black') # , weight='semibold'


In [ ]:
dfg = pd.DataFrame(txt_astro.groupby('class')['tokens'].apply(list))
dfg.columns = ['tokens']

In [ ]:
# dfg.head(8), type(dfg)

In [ ]:
dfg['tokens'] = dfg['tokens'].apply(lambda x : most_frequent_tokens(x))

In [ ]:
### https://github.com/mwouts/itables/blob/main/docs/advanced_parameters.md
show(dfg, classes="display", scrollY="400px", 
     scrollCollapse=True, paging=False, column_filters="footer", dom="lrtip")

In [ ]:
l_dct= []
for i in dct:
    for v in dct:
        if i < v:
            l_dct.append([i,v])
l_dct[:5]            

In [ ]:
G = nx.Graph()
G.add_edges_from(l_dct)

In [ ]:
pos = nx.spring_layout(G)
fig = plt.figure(figsize = (30,20))
fig.tight_layout()
nx.draw_networkx(G, pos, node_size=10)

In [ ]:
### 
# https://pyvis.readthedocs.io/en/latest/tutorial.html#using-pyvis-within-jupyter-notebook
nt = Network( notebook=True) #, filter_menu=True)
# populates the nodes and edges data structures
nt.from_nx(G)
nt.show('pictures/nx.html' )

In [ ]:
def prepare_graph(val):
    dct = {val}
    